# This is my initial data analysis

Firstly import all the relevant modules.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
train_identity = pd.read_csv("Data/initial/train_identity.csv")
train_transaction = pd.read_csv("Data/initial/train_transaction.csv")

In [ ]:
train_identity.head()

In [ ]:
train_transaction.head()

In [ ]:
train_identity.info()

In [ ]:
train_transaction.info()

In [ ]:
#for col, values in train_transaction.iteritems():
#    num_uniques = values.nunique()
#    print ('{name}: {num_unique}'.format(name=col, num_unique=num_uniques))
#    print ('\n')
#    print (values.unique())

ok for now lets just ignore the vesta features.


In [ ]:
cols = []


for col in train_transaction.columns:
    if col[0] != "V":
        cols.append(col)

print(cols)

transaction = train_transaction[cols]

transaction.head()

Now lets remove columns with lots of null values

In [ ]:
transaction.isna().sum() / len(transaction)

Some columns have almost 90% (or over!) null values so remove these columns.

In [ ]:
criteria = transaction.isna().sum() / len(transaction) > 0.85

transaction.drop(criteria.index[criteria], inplace=True, axis=1)

In [ ]:
transaction.head()

It looks like the MX columns are boolean that are masked with T=true and F=False,
so cast them to integers with 1 = True, 0=False.

In [ ]:
#Take a look at the unique items in each col
for col, values in transaction.iloc[:,38:47].iteritems():
    num_uniques = values.nunique()
    print ('{name}: {num_unique}'.format(name=col, num_unique=num_uniques))
    print (values.unique())
    print ('\n')

In [ ]:
# ok so only M4 not 'T' and 'F'.

transaction.iloc[:,38:] = transaction.iloc[:,38:].replace(to_replace="T", value=1)
transaction.iloc[:,38:] = transaction.iloc[:,38:].replace(to_replace="F", value=0)

transaction.head()

ok so seems to have worked well.

In [ ]:
transaction.info()

In [ ]:
transaction.select_dtypes(include='object').nunique()

Let us have a look at the frequencies of each email domain.

In [ ]:
transaction['P_emaildomain'].hist()

Ok so it's clear from this graph that the vast majority of emails are from a few domains.

In [ ]:
transaction['P_emaildomain'].value_counts()

Ok so let's just concatenate each domain with <5000 appearances into 'other'.

In [ ]:
P_emails = transaction['P_emaildomain'].copy()

criteria = P_emails.value_counts() > 5000

valid_emails = criteria.index[criteria]

P_emails != valid_emails.any()

P_emails[~(P_emails.isin(valid_emails))] = 'other'


In [ ]:
P_emails.unique()

In [ ]:
P_emails.value_counts().plot(kind='bar')

Now let's have a quick look at r-emails

In [ ]:
R_emails = transaction['R_emaildomain'].copy()

R_emails.value_counts() # seems to be mainly the same emails as before so use the same list as previous.

In [ ]:
criteria = R_emails.value_counts() > 5000

valid_emails = criteria.index[criteria]

R_emails != valid_emails.any()

R_emails[~(R_emails.isin(valid_emails))] = 'other'

In [ ]:
transaction['P_emaildomain'] = P_emails
transaction['R_emaildomain'] = R_emails

In [ ]:
list(valid_emails)

Now let's take a look at the unique values again.

In [ ]:
transaction.select_dtypes('object').nunique() # much more manageable

In [ ]:
# Let's one-hot encode these values now

encoded_transactions = pd.get_dummies(transaction)

Now let's take a look at the identity dataframe.

In [ ]:
identity = train_identity.copy()

In [ ]:
identity.head()

Let us again get rid of the columns that are majority null 

In [ ]:
identity.isna().sum()/len(identity)

In [ ]:
criteria = identity.isna().sum() / len(identity) > 0.85

identity.drop(criteria.index[criteria], inplace=True, axis=1)

In [ ]:
identity.head() # seems like id_35 -38 are true or false, so replace with 1 or 0.

In [ ]:
identity[['id_35','id_36','id_37','id_38']] = identity[['id_35','id_36','id_37','id_38']].replace(to_replace={'T':1, 'F':0})

In [ ]:
identity.select_dtypes('object').nunique()

Let's take a closer look at id_30-33 and device info

In [ ]:
tricky_ids = identity[['id_30','id_31','id_33','DeviceInfo']]

tricky_ids.head()

Ok so it looks like id_30 is the os and version, id_31 is the browser, id_33 is the screen dimensions and DeviceInfo is the general OS.
Let us again see if we can do the 'other' trick from before.

In [ ]:
criteria = [tricky_ids[col].value_counts() > 0.10 * max(tricky_ids[col].value_counts()) for col in tricky_ids]

criteria

In [ ]:
for cr in criteria:
    print(list(cr.index[cr]))

In [ ]:
for col, cr in zip(['id_30','id_31','id_33','DeviceInfo'], criteria):
    valid_types = cr.index[cr]
        
    ser = tricky_ids[col]
    
    ser[~(ser.isin(valid_types))] = 'other'
    
    print(ser.nunique())
    
    tricky_ids.loc[:,col] = ser
    
    
tricky_ids.loc[:,'id_30'].unique()

In [ ]:
identity[['id_30','id_31','id_33','DeviceInfo']] = tricky_ids

Now join the two dbs together

In [ ]:
full = transaction.join(identity.set_index('TransactionID'), on='TransactionID')

full.head()

In [ ]:
full.info()

In [ ]:
len(identity)

In [ ]:
(full.notna().sum(axis=1) < 39).sum()

In [ ]:
full.info()

In [ ]:
#split into numerical and cat

full_numeric = full.select_dtypes(include=np.number)
full_cat = full.select_dtypes(exclude=np.number)
full_cat.loc[:,'isFraud'] = full.loc[:,'isFraud']

full_cat = pd.get_dummies(full_cat)

full.head()

In [ ]:
corr_num = full_numeric.corr()
corr_cat = full_cat.corr()

In [ ]:
corr_num['isFraud'].sort_values

In [ ]:
corr_cat['isFraud']

In [ ]:
np.abs(corr_num['isFraud'].drop('isFraud')).plot()

# too many vars

In [ ]:
# Find vars of over 0.08

num_vars = np.abs(corr_num['isFraud'][2:]) > 0.08


num_vars[num_vars]


In [ ]:
vars_to_include = list(num_vars[num_vars].index)

In [ ]:
np.abs(corr_cat['isFraud'].drop('isFraud')).plot()

#possibly remove vars that have < 0.08 corr

In [ ]:
train_transaction['addr2'].value_counts()

In [ ]:
cat_vars = np.abs(corr_cat['isFraud'][2:]) > 0.08

cat_vars[cat_vars]

In [ ]:
vars_to_include += [list(cat_vars[cat_vars].index)]

In [ ]:
import seaborn as sns

fig, ax = plt.subplots()

sns.heatmap(corr_num)

In [ ]:
sns.heatmap(corr_cat)

Implement an hour var, by using TransactionDT

In [ ]:
transactionDT = transaction['TransactionDT']

transactionDT.head()

In [ ]:
# it is in the form of seconds from a point so divide by 60*60 (3600) to get the hour of each.

In [ ]:
transactionDT = (transactionDT - transactionDT[0]) / 3600 # convert ot fraction of an hour.

transactionDT = transactionDT.astype(int) # just take int at the front

transactionDT = transactionDT % 24 # get the hour of the day

transactionDT.unique() # looks good

transactionDT.head()

In [ ]:
# add in isFraud col and check correlation.
check_corr_df = pd.concat([transactionDT, transaction['isFraud']], axis=1)

check_corr = check_corr_df.corr()

check_corr

In [ ]:
take_only_frauds = check_corr_df[check_corr_df['isFraud'] == 1]

take_only_frauds['TransactionDT'].plot(kind='hist') # possibly could just match the pattern of business hours

Now let's take a look at the correlation matrix for these, including transactionHour

In [ ]:
list(cat_vars[cat_vars].index)

In [ ]:
vars_to_include_initial = list(num_vars[num_vars].index) + [
 'id_15',
 'id_16',
 'id_28',
 'id_29',
 'id_30',
 'id_33',
 'id_34',
 'DeviceInfo',
 'ProductCD',
 'P_emaildomain',
 'M4']

In [ ]:
full = train_transaction.merge(train_identity, on='TransactionID', how='left')

final = full[[
 'isFraud',
 'TransactionID',
 'id_17',
 'id_35',
 'TransactionDT',
 'card3',
 'id_15',
 'id_16',
 'id_28',
 'id_29',
 'id_30',
 'id_33',
 'DeviceInfo',
 'ProductCD',
 'P_emaildomain']]

#rearrange cols to put isFraud first

In [ ]:
corr_f = pd.get_dummies(final)

corr_f = corr_f.corr()

np.abs(corr_f['isFraud'][1:])

In [ ]:
len(final)

In [ ]:
1 - final.notna().sum() / len(final) # percentage of nulls.

In [ ]:
final.head()

In [ ]:
final.columns.to_list()

#rearrange cols to put isFraud first

final.head()

In [ ]:
# save final dataset to csv.

final.to_csv('Data/finaldataset.csv')

In [ ]:
final['ProductCD'].unique()